In [23]:
import pandas as pd
import numpy as np
import json
import altair as alt
import os
#os.chdir("..")


In [24]:

dfs = pd.read_excel("data/ons_subnational_2024.xlsx", None, skiprows=0)

cleanded_dfs = []
for df in dfs.values():
    # itterate down the rows until a row with 'Area Code' in the first column is found
    # this is the start of the data
    for i in range(len(df)):
        if df.iloc[i, 0] == 'Area Code':
            df.columns = df.iloc[i]
            df = df.iloc[i+1:]
            cleanded_dfs.append(df)
            break

df = pd.concat(cleanded_dfs)

id_vars = ['Area Code', 'Country', 'Nation', 'Region',
       'County or Unitary Authority', 'Local Authority District', 'Lower 95% Confidence Interval',
       'Upper 95% Confidence Interval', 'Notes', 'ITL Level 1',
       'ITL Level 2', 'ITL Level 3',
       'Data accuracy']


df = df.melt(id_vars=id_vars, var_name='Series', value_name='Value')

# if Local Authority District is undefined, use County or Unitary Authority
df['Local Authority District'] = df['Local Authority District'].fillna(
    df['County or Unitary Authority'])


df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df.dropna(subset=['Local Authority District', 'Value'])

df['median'] = df.groupby(['Series'])['Value'].transform('median')
df['std'] = df.groupby(['Series'])['Value'].transform('std')
df['z_score'] = (df['Value'] - df['median']) / df['std']

df['norm_value'] = df.Value/df['median']

series_to_keep = {
      #  'Gross Annual Pay' : {
      #           "Theme": "Income",
      #           "name": "Gross Annual Pay (2023, Full Time Only)",
      #           "format": "$,.2f",
      #           "id": "gross_annual_pay"
      #     },
    "Gross Value Added (GVA) per hour worked (£)": {
              "Theme": "Productivity",
              "name": "GVA per hour worked (£)",
              "format": "$,.2f",
              "scheme": "interpolateWarm",
                "id": "gva_per_hour",
                  "geo_version": "2022"
       },
    'Gross median weekly pay (£)': {
              "Theme": "Income",
              "name": "Gross median weekly pay (£)",
              "format": "$,.2f",
                "id": "gross_median_weekly_pay",
                "geo_version": "2023"
       },
    'Employment rate, aged 16 to 64 years (%)': {
              "Theme": "Employment",
              "name": "Employment rate, ages 16-64 (%)",
              "format": ".2%",
                "id": "employment_rate",
                  "geo_version": "2022"
       },
    'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)': {
              "Theme": "Transport",
              "name": "Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)",
              "format": ".2f",
                "id": "average_travel_time",
                  "geo_version": "2011"
       },
    'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)': {
              "Theme": "Education",
              "name": "Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)",
              "format": ".2%",
                "id": "gcse_achievements",
                "geo_version": "2022"
       },
    # 'Female Healthy Life Expectancy (years)': {
    #           "Theme": "Health",
    #           "name": "Female Healthy Life Expectancy (years)",
    #           "format": ".2f",
    #           "id": "life_expectancy_female",
    #            "geo_version": "2022"
    #    },
    # 'Male Healthy Life Expectancy (years)': {
    #     "Theme": "Health",
    #     "name": "Male Healthy Life Expectancy (years)",
    #    "format": ".2f",
    #     "id": "life_expectancy_male",
    #      "geo_version": "2022"
    # },
    'Age-standardised mortality rate for those aged under 75 (per 100,000 population)': {
              "Theme": "Health",
              "name": "Under 75 Age-standardised mortality rates",
              "format": ".2f",
                "id": "mortality_rate",
                "geo_version": "2023"
       },
    'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)': {
              "Theme": "Wellbeing",
              "name": "Mean Reported Life Satisfaction (0-10)",
              "format": ".2f",
                "id": "life_satisfaction",
                  "geo_version": "2022"
       },
    'Net additions per 1,000 stock': {
              "Theme": "Housing",
              "name": "Net additions per 1,000 stock",
              "format": ".2f",
                  "id": "net_additions",
                     "geo_version": "2022"
       },
       "Percentage of 4G coverage by at least one mobile network operator (%)" : {
              "Theme": "Digital",
              "name": "4G Coverage (%)",
              "format": ".2%",
                "id": "4g_coverage",
                  "geo_version": "2022"
       },
       "Percentage of schools rated good or outstanding by Ofsted (%)" : {
              "Theme": "Education",
              "name": "'Good' or 'Outstanding' Schools (%)",
              "format": ".2%",
                "id": "offsted_schools",
                  "geo_version": "2023"
       },
       'Percentage of adults that currently smoke cigarettes (%)':  {
              "Theme": "Health",
              "name": "Adult Smoking Rate (%)",
              "format": ".2%",
                "id": "smoking_rate",
                  "geo_version": "2022"
       },

}

df = df[df.Series.isin(series_to_keep.keys())]

df = df[['Area Code', 'Local Authority District', 'Series', 'Value', 'norm_value']]
df = df.rename(columns={'Area Code': 'lad_id', 'Local Authority District': 'lad_name',
               'Series': 'series', 'Value': 'value', 'norm_value': 'norm_value'})

# add the series id
df['series_id'] = df['series'].apply(lambda x: series_to_keep[x]['id'])

tool_data = {}
indicator_data = {}

for series_id in df['series_id'].unique():
    series_data = df[df['series_id'] == series_id][['lad_id', 'lad_name', 'value', 'norm_value']].dropna(
    ).reset_index(drop=True).to_dict(orient='records')
    indicator_data[series_id] = {
         "data": series_data,
         "meta": series_to_keep[df[df['series_id'] == series_id]['series'].iloc[0]]
    }

tool_data['indicators'] = indicator_data

indicator_data

tool_data['geo'] = {}
# add the LAD_2011, LAD_2022, LAD_2023 .json from data
for year in ['2011', '2022', '2023']:
   with open(f'data/LAD_{year}.json') as f:
      geo_data = json.load(f)
        # rename LADXXCD to lad_id
      for feature in geo_data['features']:
            feature['properties']['lad_id'] = feature['properties']['LAD'+year[-2:]+'CD'] if year != '2011' else feature['properties']['lad11cd']
            feature['properties']['lad_name'] = feature['properties']['LAD'+year[-2:]+'NM'] if year != '2011' else feature['properties']['lad11nm']
      tool_data['geo'][year] = geo_data
    
    
tool_data

with open('data/indicators.json', 'w') as f:
    json.dump(tool_data, f)

In [19]:
# is oxford in there? incl. substring
df.query("series == 'Male Healthy Life Expectancy (years)'")

,lad_id,lad_name,series,value,norm_value,series_id


In [8]:
with open("temp.json", "w") as f:
    json.dump(tool_data['indicators']['gross_median_weekly_pay']['data'], f)

In [4]:
    'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)': {
              "Theme": "Transport",
              "name": "Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)",
              "format": ".2f",
                "id": "average_travel_time",
                  "geo_version": "2011"
       },
    'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)': {
              "Theme": "Education",
              "name": "Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)",
              "format": ".2%",
                "id": "gcse_achievements",
                "geo_version": "2022"
       },
    'Female Healthy Life Expectancy (years)': {
              "Theme": "Health",
              "name": "Female Healthy Life Expectancy (years)",
              "format": ".2f",
              "id": "life_expectancy_female",
               "geo_version": "2022"
       },
    'Male Healthy Life Expectancy (years)': {
        "Theme": "Health",
        "name": "Male Healthy Life Expectancy (years)",
       "format": ".2f",
        "id": "life_expectancy_male",
         "geo_version": "2022"
    },
    'Age-standardised mortality rate for those aged under 75 (per 100,000 population)': {
              "Theme": "Health",
              "name": "Under 75 Age-standardised mortality rates",
              "format": ".2f",
                "id": "mortality_rate",
                "geo_version": "2023"
       },
    'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)': {
              "Theme": "Wellbeing",
              "name": "Mean Reported Life Satisfaction (0-10)",
              "format": ".2f",
                "id": "life_satisfaction",
                  "geo_version": "2022"
       },
    'Net additions per 1,000 stock': {
              "Theme": "Housing",
              "name": "Net additions per 1,000 stock",
              "format": ".2f",
                  "id": "net_additions",
                     "geo_version": "2022"
       }
}

df = df[df.Series.isin(series_to_keep.keys())]

df = df[['Area Code', 'Local Authority District', 'Series', 'Value', 'norm_value']]
df = df.rename(columns={'Area Code': 'lad_id', 'Local Authority District': 'lad_name',
               'Series': 'series', 'Value': 'value', 'norm_value': 'norm_value'})

# add the series id
df['series_id'] = df['series'].apply(lambda x: series_to_keep[x]['id'])

tool_data = {}
indicator_data = {}

for series_id in df['series_id'].unique():
    series_data = df[df['series_id'] == series_id][['lad_id', 'value', 'norm_value']].dropna(
    ).reset_index(drop=True).to_dict(orient='records')
    indicator_data[series_id] = {
         "data": series_data,
         "meta": series_to_keep[df[df['series_id'] == series_id]['series'].iloc[0]]
    }

tool_data['indicators'] = indicator_data

indicator_data

tool_data['geo'] = {}
# add the LAD_2011, LAD_2022, LAD_2023 .json from data
for year in ['2011', '2022', '2023']:
   with open(f'data/LAD_{year}.json') as f:
      geo_data = json.load(f)
        # rename LADXXCD to lad_id
      for feature in geo_data['features']:
            feature['properties']['lad_id'] = feature['properties']['LAD'+year[-2:]+'CD'] if year != '2011' else feature['properties']['lad11cd']
      tool_data['geo'][year] = geo_data
    
    
tool_data

with open('data/indicators.json', 'w') as f:
    json.dump(tool_data, f)

SyntaxError: unmatched '}' (3944276549.py, line 50)

In [ ]:
    'Average travel time in minutes to reach nearest large employment centre (500 to 4999 jobs available), by public transport or walking (minutes)': {
              "Theme": "Transport",
              "name": "Average Public Transport Travel Time to Nearest Large Employment Centre (minutes)",
              "format": ".2f",
                "id": "average_travel_time",
                  "geo_version": "2011"
       },
    'Percentage of young people achieving GCSEs (and equivalent qualifications) in English and maths by age 19 (%)': {
              "Theme": "Education",
              "name": "Percentage of Young People Achieving GCSEs in English and Maths by Age 19 (%)",
              "format": ".2%",
                "id": "gcse_achievements",
                "geo_version": "2022"
       },
    'Female Healthy Life Expectancy (years)': {
              "Theme": "Health",
              "name": "Female Healthy Life Expectancy (years)",
              "format": ".2f",
              "id": "life_expectancy_female",
               "geo_version": "2022"
       },
    'Male Healthy Life Expectancy (years)': {
        "Theme": "Health",
        "name": "Male Healthy Life Expectancy (years)",
       "format": ".2f",
        "id": "life_expectancy_male",
         "geo_version": "2022"
    },
    'Age-standardised mortality rate for those aged under 75 (per 100,000 population)': {
              "Theme": "Health",
              "name": "Under 75 Age-standardised mortality rates",
              "format": ".2f",
                "id": "mortality_rate",
                "geo_version": "2023"
       },
    'Mean satisfaction with your life nowadays scored 0 (not at all) - 10 (completely)': {
              "Theme": "Wellbeing",
              "name": "Mean Reported Life Satisfaction (0-10)",
              "format": ".2f",
                "id": "life_satisfaction",
                  "geo_version": "2022"
       },
    'Net additions per 1,000 stock': {
              "Theme": "Housing",
              "name": "Net additions per 1,000 stock",
              "format": ".2f",
                  "id": "net_additions",
                     "geo_version": "2022"
       }
}

df = df[df.Series.isin(series_to_keep.keys())]

df = df[['Area Code', 'Local Authority District', 'Series', 'Value', 'norm_value']]
df = df.rename(columns={'Area Code': 'lad_id', 'Local Authority District': 'lad_name',
               'Series': 'series', 'Value': 'value', 'norm_value': 'norm_value'})

# add the series id
df['series_id'] = df['series'].apply(lambda x: series_to_keep[x]['id'])

tool_data = {}
indicator_data = {}

for series_id in df['series_id'].unique():
    series_data = df[df['series_id'] == series_id][['lad_id', 'value', 'norm_value']].dropna(
    ).reset_index(drop=True).to_dict(orient='records')
    indicator_data[series_id] = {
         "data": series_data,
         "meta": series_to_keep[df[df['series_id'] == series_id]['series'].iloc[0]]
    }

tool_data['indicators'] = indicator_data

indicator_data

tool_data['geo'] = {}
# add the LAD_2011, LAD_2022, LAD_2023 .json from data
for year in ['2011', '2022', '2023']:
   with open(f'data/LAD_{year}.json') as f:
      geo_data = json.load(f)
        # rename LADXXCD to lad_id
      for feature in geo_data['features']:
            feature['properties']['lad_id'] = feature['properties']['LAD'+year[-2:]+'CD'] if year != '2011' else feature['properties']['lad11cd']
      tool_data['geo'][year] = geo_data
    
    
tool_data

with open('data/indicators.json', 'w') as f:
    json.dump(tool_data, f)

In [1]:
indicator_data

NameError: name 'indicator_data' is not defined

In [37]:
df.series_id.unique()

array(['gva_per_hour', 'gross_median_weekly_pay', 'average_travel_time',
       'gcse_achievements', 'life_expectancy_female',
       'life_expectancy_male', 'mortality_rate', 'life_satisfaction',
       'net_additions'], dtype=object)

In [8]:
# need LADs for 2022, 2023, 2011, 


,Area Code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes,ITL Level 1,ITL Level 2,ITL Level 3,Data accuracy,Series,Value,median,std,z_score,norm_value
0,E06000047,NaN,NaN,NaN,County Durham,County Durham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),30.64,34.275,8.183461,-0.444189,0.893946
1,E06000005,NaN,NaN,NaN,Darlington,Darlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),28.56,34.275,8.183461,-0.698360,0.833260
2,E06000001,NaN,NaN,NaN,Hartlepool,Hartlepool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),29.84,34.275,8.183461,-0.541947,0.870605
3,E06000002,NaN,NaN,NaN,Middlesbrough,Middlesbrough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),29.50,34.275,8.183461,-0.583494,0.860686
4,E06000057,NaN,NaN,NaN,Northumberland,Northumberland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gross Value Added (GVA) per hour worked (£),30.43,34.275,8.183461,-0.469850,0.887819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706510,E10000027,NaN,NaN,NaN,Somerset,Somerset,NaN,NaN,Obsolete code since 2023 (replaced by E0600006...,NaN,NaN,NaN,NaN,"Net additions per 1,000 stock",11.00,9.000,5.156428,0.387865,1.222222
706511,E07000187,NaN,NaN,NaN,NaN,Mendip,NaN,NaN,Obsolete code since 2023 (replaced by E0600006...,NaN,NaN,NaN,NaN,"Net additions per 1,000 stock",10.00,9.000,5.156428,0.193933,1.111111
706512,E07000188,NaN,NaN,NaN,NaN,Sedgemoor,NaN,NaN,Obsolete code since 2023 (replaced by E0600006...,NaN,NaN,NaN,NaN,"Net additions per 1,000 stock",12.00,9.000,5.156428,0.581798,1.333333
706513,E07000246,NaN,NaN,NaN,NaN,Somerset West and Taunton,NaN,NaN,Obsolete code since 2023 (replaced by E0600006...,NaN,NaN,NaN,NaN,"Net additions per 1,000 stock",13.00,9.000,5.156428,0.775731,1.444444
